In [19]:
import pandas as pd
import numpy as np
import os
try:
    import arcpy
except ImportError:
    pass

OVERWRITE = True
PATH = os.path.join('.', 'data')

# TODO: There's some data that is inclued as a sort of separate table with the .xlsx file, that we are just throwing away at present
# TODO: Handle combined precincts
# TODO: Delete superfluous fields from derived_data.gdb/blocks

In [20]:
def to_appropriate_column_name(s):
    s = s.lower().replace(' ', '_')
    s = ''.join([ch for ch in s if ch.isalnum() or ch == '_'])
    return s

# Tables
## Clean the population data

In [16]:
pop_df = pd.read_excel(os.path.join(PATH, 'source_data', '2010_Pop_Block_County.xls'), sheetname='San Francisco County', header=4)

pop_df = pd.read_excel(os.path.join(PATH, 'source_data', '2010_Pop_Block_County.xls'), sheetname='San Francisco County', header=4)
pop_df = pop_df.drop('BLOCKS', axis='index').reset_index()
pop_df = pop_df.rename(columns={'index': 'block_str'})

records = []
for ix, row in pop_df.iterrows():
    splits = row['block_str'].split(', ')
    record = row[[x for x in row.index if x != 'block_str']].to_dict()
    record['block'] = str(splits[0].split('Block ')[-1])
    record['block_group'] = str(splits[1].split('Block Group ')[-1])
    census_tract = '0' + str(splits[2].split('Census Tract ')[-1].replace('.', ''))
    census_tract += '0' * (6-len(census_tract))  # Even if it doesn't have a decimal part, needs to be 6 characters
    record['census_tract'] = census_tract
    records.append(record)
pop_df = pd.DataFrame(records)

pop_df.columns = [to_appropriate_column_name(x) for x in pop_df.columns]
str_columns = ['block', 'block_group', 'census_tract']
nonstr_columns = [x for x in pop_df.columns if x not in str_columns]
pop_df.loc[:, nonstr_columns] = pop_df[nonstr_columns].astype(int)
pop_df = pop_df[str_columns + nonstr_columns]
pop_df.to_csv(os.path.join(PATH, 'derived_data', 'SF_2010_pop_block.csv'))
pop_df.head()

,block,block_group,census_tract,american_indian_and_alaska_native,asian,black_or_african_american,hispanic_or_latino,native_hawaiian_and_other_pacific_islander,one_race_total,some_other_race,total_population,total_population_not_hispanic_or_latino,two_or_more_races,white
0,1000,1,010100,0,0,0,0,0,0,0,0,0,0,0
1,1001,1,010100,0,4,3,1,0,44,1,44,43,0,36
2,1002,1,010100,0,0,0,0,0,0,0,0,0,0,0
3,1003,1,010100,0,0,0,0,0,0,0,0,0,0,0
4,1004,1,010100,0,0,1,0,0,1,0,1,1,0,0


## Clean and split the election data

In [21]:
vote_dfs = pd.read_excel(os.path.join(PATH, 'source_data', '20161206_sov.xlsx'), sheetname=None, header=3)

# Since the sheet names get cut off, we can fix them using the Contents tab
contents = vote_dfs['Contents'].copy()
contents.columns = ['key', 'name']
contents = contents.iloc[1:]
contents['key'] = contents['key'].astype(int)
contents = contents.set_index('key').to_dict(orient='index')

fixed_names = {}
for cut_name in vote_dfs:
    if cut_name == 'Contents':
        continue
    key = int(cut_name[:3])
    if key in contents:
        prefix = cut_name[:6]
        postfix = contents[key]['name']
        fixed_names[prefix + postfix] = vote_dfs[cut_name]

In [22]:
fixed_names.values()[0].columns

Index([u'PrecinctName', u'ReportingType', u'PrecinctID', u'Precincts',
       u'Registration', u'Ballots Cast', u'Turnout (%)', u'Yes', u'No',
       u'Under Vote', u'Over Vote'],
      dtype='object')

In [23]:
for name, df in fixed_names.items():
    if name == 'Contents':
        continue
    df = df[df['PrecinctName'].apply(lambda x: str(x).startswith('Pct '))]
    df.columns = [to_appropriate_column_name(x.replace('(%)', 'Percent')) for x in df.columns]
    df.to_csv(os.path.join(PATH, 'derived_data', '{}.csv'.format(name)), encoding='utf-8', index=False)
    #####
    agg_methods = {
    'precincts': 'min',
    'registration': 'min'
    }
    no_agg = ['OBJECTID', 'precinctname', 'reportingtype', 'precinctid', 'turnout_percent']
    grouped_df = df.groupby('precinctid').agg({x: agg_methods.get(x, 'sum') for x in df.columns if x not in no_agg})
    grouped_df.to_csv(os.path.join(PATH, 'derived_data', '{}.csv'.format('precinct_summary_' + name)), encoding='utf-8', index=True)

## Make a geodatabase from the election CSVs

In [6]:
if OVERWRITE:
    arcpy.Delete_management(os.path.join(PATH, 'SF_vote_2016.gdb'))
arcpy.CreateFileGDB_management(PATH, 'SF_vote_2016.gdb')
arcpy.TableToGeodatabase_conversion(
    [os.path.join(PATH, 'derived_data', '{}.csv'.format(x)) for x in fixed_names.keys()],
    os.path.join(PATH, 'SF_vote_2016.gdb'))

<Result 'C:\\Users\\Charles\\Documents\\ArcGIS\\SF_election_2016\\SF_vote_2016.gdb'>

In [7]:
arcpy.env.workspace = os.path.join(PATH, 'SF_vote_2016.gdb')

NO_TOUCH = ['OBJECTID', 'precinctname', 'reportingtype', 'precinctid']
FLOAT = ['turnout_percent']

for table in sorted(arcpy.ListTables()):
    for field in arcpy.ListFields(table):
        if field.name not in NO_TOUCH:
            original_name = field.name
            temp_name = field.name[:5] + '_temp'
            arcpy.AddField_management(table, temp_name, 'FLOAT' if original_name in FLOAT else 'LONG')
            arcpy.CalculateField_management(table, temp_name, u'!{}!'.format(original_name), "PYTHON_9.3")
            arcpy.DeleteField_management(table, original_name)
            arcpy.AlterField_management(table, temp_name, to_appropriate_column_name(original_name[:31]))
    print('{} complete'.format(table))

T100___President_and_Vice_President complete
T105___U_S__Senator complete
T110___U_S__Representative__District_12 complete
T115___U_S__Representative__District_13 complete
T120___U_S__Representative__District_14 complete
T125___State_Senate__District_11 complete
T130___State_Assembly__District_17 complete
T135___State_Assembly__District_19 complete
T140___Board_of_Supervisors__District_1 complete
T145___Board_of_Supervisors__District_3 complete
T150___Board_of_Supervisors__District_5 complete
T155___Board_of_Supervisors__District_7 complete
T160___Board_of_Supervisors__District_9 complete
T165___Board_of_Supervisors__District_11 complete
T170___Superior_Court_Judge__Seat_7 complete
T175___Member__Board_of_Education complete
T180___Member__Community_College_Board complete
T185___BART_Director__District_7 complete
T190___BART_Director__District_9 complete
T195___State_Proposition_51 complete
T200___State_Proposition_52 complete
T205___State_Proposition_53 complete
T210___State_Propositio

# Feature Classes
## Spatial join blocks to precincts

In [9]:
if OVERWRITE:
    arcpy.Delete_management(os.path.join(PATH, 'source_data.gdb'))
    arcpy.Delete_management(os.path.join(PATH, 'derived_data.gdb'))
arcpy.CreateFileGDB_management(PATH, 'source_data.gdb')
arcpy.CreateFileGDB_management(PATH, 'derived_data.gdb')
arcpy.FeatureClassToGeodatabase_conversion(
    [
        os.path.join(PATH, 'source_data', 'Census 2010- Blocks for San Francisco', 'geo_export_72486aab-d116-414f-895d-9a7fbaa2a42c.shp'),
        os.path.join(PATH, 'source_data', '2012lines', '2012lines', 'SF_DOE_Precincts_20120702.shp'),
        os.path.join(PATH, 'source_data', '2012lines', '2012lines', 'SF_BOS_20120702_nowater.shp'),
    ],
    os.path.join(PATH, 'source_data.gdb'))
arcpy.TableToGeodatabase_conversion(
    os.path.join(PATH, 'derived_data', 'SF_2010_pop_block.cv'),
    os.path.join(PATH, 'derived_data.gdb')
)

# We don't use a field mapping here because we actually want all the fields from both the target and join inputs
arcpy.SpatialJoin_analysis(
    os.path.join(PATH, 'source_data.gdb', 'geo_export_72486aab_d116_414f_895d_9a7fbaa2a42c'),
    os.path.join(PATH, 'source_data.gdb', 'SF_DOE_Precincts_20120702'),
    os.path.join(PATH, 'derived_data.gdb', 'blocks_join_precincts'),
    match_option='WITHIN',
)

# Since some blocks don't fit in a precicnt, we at least want them to have a distict

fms = arcpy.FieldMappings()
for field in arcpy.ListFields(os.path.join(PATH, 'derived_data.gdb', 'blocks_join_precincts')):
    if field.name.lower() == 'shape':
        continue
    map = arcpy.FieldMap()
    map.addInputField(os.path.join(PATH, 'derived_data.gdb', 'blocks_join_precincts'), field.name)
    fms.addFieldMap(map)
map = arcpy.FieldMap()
map.addInputField(os.path.join(PATH, 'source_data.gdb', 'SF_BOS_20120702_nowater'), 'DISTRICT')
fms.addFieldMap(map)

arcpy.SpatialJoin_analysis(
    os.path.join(PATH, 'derived_data.gdb', 'blocks_join_precincts'),
    os.path.join(PATH, 'source_data.gdb', 'SF_BOS_20120702_nowater'),
    os.path.join(PATH, 'derived_data.gdb', 'pop_by_block'),
    match_option='WITHIN',
    field_mapping=fms
)

arcpy.AlterField_management(os.path.join(PATH, 'derived_data.gdb', 'pop_by_block'), 'Join_Count', 'join_count_precinct')
arcpy.AlterField_management(os.path.join(PATH, 'derived_data.gdb', 'pop_by_block'), 'Join_Count_1', 'join_count_district')
arcpy.Delete_management(os.path.join(PATH, 'derived_data.gdb', 'blocks_join_precincts'))

<Result 'true'>

## Join population data to blocks

In [10]:
# First we have to make a precinct_id column, since PREC_2012 is text
# arcpy.AddField_management(PATH + '\\derived_data.gdb\\pop_by_block', 'precinct_id', 'LONG')
# arcpy.CalculateField_management(PATH + '\\derived_data.gdb\\pop_by_block', 'precinct_id', '!PREC_2012!', "PYTHON_9.3")

# Create join key columns, this shit is so jenky
arcpy.AddField_management(os.path.join(PATH, 'derived_data.gdb', 'pop_by_block'), 'pop_block_join_key', 'TEXT')
arcpy.CalculateField_management(
    os.path.join(PATH, 'derived_data.gdb', 'pop_by_block'),
    'pop_block_join_key',
    'str(!tractce10!) + str(!blockce10!)',
    'PYTHON_9.3'
)
arcpy.AddField_management(os.path.join(PATH, 'derived_data.gdb', 'SF_2010_pop_block'), 'pop_block_join_key', 'TEXT')
arcpy.CalculateField_management(
    os.path.join(PATH, 'derived_data.gdb', 'SF_2010_pop_block'),
    'pop_block_join_key',
    "'0' + str(!census_tract!) + str(!block!)",
    'PYTHON_9.3'
)

# Now do the actual join
arcpy.JoinField_management(
    os.path.join(PATH, 'derived_data.gdb', 'pop_by_block'),
    'pop_block_join_key',
    os.path.join(PATH, 'derived_data.gdb', 'SF_2010_pop_block'),
    'pop_block_join_key'
)

# Also make a precinct_id column, since PREC_2012 is text
arcpy.AddField_management(os.path.join(PATH, 'derived_data.gdb', 'pop_by_block'), 'precinct_id', 'LONG')
func = """def to_int(x):
    if sum([a.isalnum() for a in x]) == 0:
        return None
    else:
        return int(x)
    """
arcpy.CalculateField_management(
    os.path.join(PATH, 'derived_data.gdb', 'pop_by_block'),
    'precinct_id',
    'to_int(!PREC_2012!)',
    'PYTHON_9.3',
    func
)

<Result 'C:\\Users\\Charles\\Documents\\ArcGIS\\SF_election_2016\\derived_data.gdb\\pop_by_block'>

In [11]:
agg_methods = {
    'SupDist': 'MIN',
    'BARTDist': 'MIN',
    'AssemDist': 'MIN',
    'CongDist': 'MIN',
    'NeighRep': 'MIN',
    'PREC_2012': 'MIN',
    'PREC_2010': 'MIN',
    'DISTRICT': 'MIN',
    'american_indian_and_alaska_native': 'SUM',
    'asian': 'SUM',
    'black_or_african_american': 'SUM',
    'hispanic_or_latino': 'SUM',
    'native_hawaiian_and_other_pacific_islander': 'SUM',
    'one_race_total': 'SUM',
    'some_other_race': 'SUM',
    'total_population': 'SUM',
    'total_population_not_hispanic_or_latino': 'SUM',
    'two_or_more_races': 'SUM',
    'white': 'SUM'
}


if OVERWRITE:
    arcpy.Delete_management(os.path.join(PATH, 'derived_data.gdb', 'pop_by_precinct'))
arcpy.Statistics_analysis(
    os.path.join(PATH, 'derived_data.gdb', 'pop_by_block'),
    os.path.join(PATH, 'derived_data.gdb', 'pop_by_precinct'),
    agg_methods.items(),
    'precinct_id'
)

for field in arcpy.ListFields(os.path.join(PATH, 'derived_data.gdb', 'pop_by_precinct')):
    if field.name[:4] in ('SUM_', 'MIN_'):
        arcpy.AlterField_management(os.path.join(PATH, 'derived_data.gdb', 'pop_by_precinct'), field.name, field.name[4:35])
arcpy.AlterField_management(os.path.join(PATH, 'derived_data.gdb', 'pop_by_precinct'), 'FREQUENCY', 'num_blocks')

<Result 'C:\\Users\\Charles\\Documents\\ArcGIS\\SF_election_2016\\derived_data.gdb\\pop_by_precinct'>